In [ ]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing

In [ ]:
from tensorflow.keras import preprocessing, utils
import os
import yaml

The dataset contains .yml files which have pairs of different questions and their answers on varied subjects like history, bot profile, science etc.
We can easily read them as folows:

In [ ]:
dir_path = '/content/sample_data'
files_list = [f for f in os.listdir(dir_path) if f.endswith('.yml')]

In [ ]:
questions = list()
answers = list()
for filepath in files_list:
    stream = open( dir_path  +"/"+ filepath , 'rb')
    # print(stream)
    docs = yaml.safe_load(stream)
    print(docs)
    
    conversations = docs['conversations']
    for con in conversations:
      if len( con ) > 2 :
        questions.append(con[0])
        replies = con[ 1 : ]
        ans = ''
        for rep in replies:
          if isinstance(rep,list):
            for i in rep:
              ans += ' ' + i
              answers.append( ans )
          else:
              ans += ' ' + rep
              answers.append( ans )

      elif len( con )> 1:
        questions.append(con[0])
        answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
  if type( answers[i] ) == str:
    answers_with_tags.append( answers[i] )
  else:
    questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
  answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
print(answers)
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


[['\ufeffتم جواب دے سکتے ہو؟', 'میں معزرت خواہ ہوں۔'], ['آپ کا نام کیا ہے؟', 'میرا نام چیٹ باٹ ہے۔'], ['تم کون ہو؟', 'میں باٹ ہوں۔'], ['کیا تم روباٹ ہو؟', 'میں روباٹ کی طرح ہوں۔'], ['آپ کی عمر کتنی ہے؟', 'میری عمر ایک سال ہے۔'], ['آپ کی پیدائش کیا ہے؟', 'میں۲۰۱۹ کو پیدا ہوا تھا۔'], ['آپ کا برج کیا ہے؟', 'میرا برج بھیڑیا ہے۔'], ['آپ کا پسندیدہ رنگ کون سا ہے؟', 'میرا پسندیدہ رنگ سفید ہے۔'], ['آپ کا لکی نمبر کون سا ہے؟', 'میرا لکی نمبر 7 ہے۔'], ['کیا آپ کے پاس کوئی پالتو جانور ہے؟', 'جی ہاں میرے پاس ایک بلی ہے۔'], ['آپ کہاں رہتے ہو؟', 'میں ایبٹ آباد میں رہتا ہوں۔'], ['آپ کیا کر رہے ہیں؟', 'میں بات کر رہا ہوں۔'], ['آپ کی لمبائی کتنی ہے؟', 'میری لمبائی 5.7 انچ ہے۔'], ['قرآن مجید کیا ہے؟', 'قرآن مجید ایک اسلامی کتاب ہے۔'], ['آپ کے جوتے کا نمبر کیا ہے؟', 'جوتے کا نمبر نوهے۔'], ['آپ کے پاس جوتوں کے کتنے جوڑے ہیں؟', 'میرے پاس جوتوں کے پانچ جوڑے ہیں۔'], ['اگر آپ دنیا کے بادشاہ ہوتے توکونسا قانون تبدیل کرتے؟', 'اگر میں دنیا کا بادشاہ ہوتا تو میں انسانیت کو ترجیح دیتا۔'], ['اگر آپ سپر ہیرو ہوتے

TypeError: ignored

### b) Preparing data for Seq2Seq model

This model requires 3 arrays encoder_input_data, decoder_input_data and decoder_output_data.

For encoder_input_data:
Tokensize the Questions and Pad them to their maximum Length.

For decoder_input_data:
Tokensize the Answers and Pad them to their maximum Length.

For decoder_output_data:
Tokensize the Answers and Remove the 1st element from all the tokenized_answers. This is the <START> element which was added earlier.

In [ ]:
from gensim.models import Word2Vec
import re

In [ ]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

In [ ]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape)

ValueError: ignored

In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print(decoder_input_data.shape , maxlen_answers )
print( encoder_input_data.shape , maxlen_answers )

ValueError: ignored

In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(0, 17, 295)


# Step 4: Defining Encoder Decoder Model





In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
#output = activation(dot(input, kernel) + bias
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

encoder_input_data.shape

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 8, 200)       10200       ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 9, 200)       10200       ['input_6[0][0]']                
                                                                                            

(8, 8)

# Step 5: Training the Model

We train the model for a number of epochs with RMSprop optimizer and categorical_crossentropy loss function.

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.h5' )

Epoch 1/150
1/1 [==============================] - 7s 7s/step - loss: 3.9299
Epoch 2/150
1/1 [==============================] - 0s 41ms/step - loss: 3.9067
Epoch 3/150
1/1 [==============================] - 0s 40ms/step - loss: 3.8863
Epoch 4/150
1/1 [==============================] - 0s 40ms/step - loss: 3.8644
Epoch 5/150
1/1 [==============================] - 0s 50ms/step - loss: 3.8380
Epoch 6/150
1/1 [==============================] - 0s 40ms/step - loss: 3.8030
Epoch 7/150
1/1 [==============================] - 0s 37ms/step - loss: 3.7521
Epoch 8/150
1/1 [==============================] - 0s 41ms/step - loss: 3.6700
Epoch 9/150
1/1 [==============================] - 0s 39ms/step - loss: 3.5272
Epoch 10/150
1/1 [==============================] - 0s 38ms/step - loss: 3.2945
Epoch 11/150
1/1 [==============================] - 0s 50ms/step - loss: 3.0908
Epoch 12/150
1/1 [==============================] - 0s 39ms/step - loss: 3.0308
Epoch 13/150
1/1 [==============================] -

# Step 6: Defining Inference Models

Encoder Inference Model: Takes questions as input and outputs LSTM states (h and c)

Decoder Inference Model: Takes in 2 inputs one are the LSTM states, second are the answer input sequences. it will o/p the answers for questions which fed to the encoder model and it's state values.

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

# Step 7: Talking with the Chatbot

define a method str_to_tokens which converts str questions to Integer tokens with padding.

1. First, we take a question as input and predict the state values using enc_model.
2. We set the state values in the decoder's LSTM.
3. Then, we generate a sequence which contains the <start> element.
4. We input this sequence in the dec_model.
5. We replace the <start> element with the element which was predicted by the dec_model and update the state values.
6. We carry out the above steps iteratively till we hit the <end> tag or the maximum answer length.



In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

1/1 [==============================] - 25s 25s/step


1/1 [==============================] - 0s 30ms/step
 ہال نزدیک یہ یہ بہت بڑی ہے۔ end


# Conversion to TFLite 

We can convert our seq2seq model to a TensorFlow Lite model so that we can use it on edge devices


In [ ]:
!pip install tf-nightly

     |████████████████████████████████| 325.2MB 28kB/s 
     |████████████████████████████████| 6.7MB 45.9MB/s 
     |████████████████████████████████| 460kB 41.8MB/s 


In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model( enc_model )
# buffer = converter.convert()
# open( 'enc_model.tflite' , 'wb' ).write( buffer )

# converter = tf.lite.TFLiteConverter.from_keras_model( dec_model )
# open( 'dec_model.tflite' , 'wb' ).write( buffer )
!pip freeze > requirements.txt